In [38]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20) 
pd.set_option('display.width', 100)

In [41]:
airr = pd.read_table('../data/snkmk_outs/combined_igblast.airr.tsv')

/home/michaelswift/miniconda3/envs/scirpy/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (93,94) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [42]:
airr.productive == 'T'

# get rid of merging artefacts
print(airr.shape[0])
airr = airr.dropna(subset=['locus'])
print(airr.shape[0], "removed presumed merging artefacts")
airr = airr[airr.productive == "T"]
print(airr.shape[0], "removed unproductive chains")

print("{} assemblies from TS".format(airr.shape[0]))

#df = pd.read_csv('notebooks/Clonality/BCRChangeo.csv', index_col=0)
df = airr
#10X
df.loc[:,"cell_id"] = df.sequence_id.replace('_contig_[0-9]', '', regex=True)
df.cell_id.replace("\-","_", regex=True, inplace=True)
# 10X
df.cell_id.replace("_5prime..CR", '_5prime', regex = True, inplace = True)
df.cell_id.replace('_[0-9]_[T][S]', '_TS',regex = True, inplace = True)
df.cell_id.replace('10X_[0-9]_[0-9]', '10X',regex = True, inplace = True)
df.loc[:, 'cell_id'] = df.cell_id.str.rsplit(" ", expand=True)[0]

df.head().cell_id.values[-20:]
# Munge the data a little
df.loc[:, 'cell_id'] = df.cell_id.str.split('|', expand=True)[0]

21079
21079 removed presumed merging artefacts
19269 removed unproductive chains
19269 assemblies from TS


## TSP metadata

In [61]:
met[(met.donor == 'TSP1') & 
    (met.method == 'smartseq2') & 
    (met.cell_ontology_class.str.contains('b cell|plasma cell'))].groupby('tissue').cell_ontology_class.value_counts()

tissue    cell_ontology_class
Bladder   plasma cell            84
          b cell                 11
Blood     naive b cell           35
          memory b cell          25
Lung      club cell               9
          b cell                  3
          plasma cell             1
Pancreas  b cell                 89
Name: cell_ontology_class, dtype: int64

In [ ]:
met[(met.donor == 'TSP1') & (met.method != '10X') & (met.cell_ontology_class.str.contains('b cell|plasma cell'))]

In [45]:
met = pd.read_csv('../data/TSP1_TSP15_metadata.csv')
#met = pd.read_csv('Immune_Obs.csv')

# yet another corner case currently not worth fixing this, maybe angela can do it
met[(met.donor == "TSP1") & (met.method == 'smartseq2')].cell_id.str.rsplit(".", n = 5, expand = True)[0]


met['_cell_id'] = met['cell_id']
met['_cell_id'] = met['_cell_id'].str.replace('_[0-2]_[0-2]_5prime', '_5prime', regex = True)
met['_cell_id'] = met['_cell_id'].str.replace('.homo.gencode.v30.ERCC.chrM', '')

/home/michaelswift/miniconda3/envs/scirpy/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-45-4e33b9203cf6>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  met['_cell_id'] = met['_cell_id'].str.replace('.homo.gencode.v30.ERCC.chrM', '')


In [46]:
merged = pd.merge(df, met, how = 'inner', left_on='cell_id', right_on='_cell_id')

In [47]:
print(merged.shape[0], "assmeblies remain after merging")

10801 assmeblies remain after merging


In [56]:
met[(met.cell_ontology_class.str.contains('b cell')) & (met.method == 'smartseq2')].tissue.value_counts()

Lymph_Node         852
Spleen             598
Blood              222
Salivary_Gland     119
Pancreas            90
Bone_Marrow         71
Kidney              67
Fat                 30
Eye                 27
Lung                20
Skin                18
Thymus              17
Bladder             16
Mammary             13
Uterus               7
Trachea              3
Liver                3
Vasculature          1
Small_Intestine      1
Name: tissue, dtype: int64

In [48]:
met.groupby('donor').tissue.value_counts()

donor  tissue         
TSP1   Lung               13211
       Bladder            11736
       Pancreas            6798
       Blood               4020
       Muscle              3760
TSP10  Fat                12768
       Skin                5875
       Blood               5028
TSP11  Bone_Marrow         2514
TSP12  Heart              11505
TSP13  Bone_Marrow          543
TSP14  Lymph_Node         30427
       Thymus             21578
       Spleen             18965
       Salivary_Gland     17858
       Lung               11971
       Large_Intestine    11439
       Small_Intestine    10458
       Blood               9964
       Vasculature         8684
       Muscle              8404
       Fat                 7696
       Bone_Marrow         6095
       Tongue              5428
       Liver               4496
       Skin                3563
       Prostate            3282
       Bladder             3066
TSP15  Eye                 5651
TSP2   Lung               18674
       Muscle    

In [49]:
met[(met['tissue'] == 'Blood') & (met['method'] =='smartseq2')].groupby('donor').cell_ontology_class.value_counts()

donor  cell_ontology_class                                        
TSP1   cd4-positive, alpha-beta t cell                                158
       naive b cell                                                    35
       cd8-positive, alpha-beta t cell                                 26
       memory b cell                                                   25
       monocyte                                                        18
       nk cell                                                         11
       erythrocyte doublet                                              5
       classical monocyte                                               4
       cd4-positive helper t cell                                       3
       intermediate monocyte                                            2
       cd24 neutrophil                                                  1
       erythrocyte                                                      1
TSP2   monocyte                              

In [50]:
merged.donor.value_counts()

TSP2     5980
TSP7     1831
TSP11     960
TSP4      698
TSP10     372
TSP1      355
TSP13     263
TSP3       94
TSP8       93
TSP6       77
TSP5       71
TSP9        7
Name: donor, dtype: int64

In [53]:
pd.DataFrame(merged.groupby(['donor', 
                'tissue']).cell_ontology_class.value_counts())

cell_ontology_class
donor tissue   cell_ontology_class                                 
TSP1  Blood    cd4-positive, alpha-beta t cell                   94
               cd8-positive, alpha-beta t cell                    7
               cd4-positive helper t cell                         2
               classical monocyte                                 2
               memory b cell                                      2
...                                                             ...
TSP7  Tongue   fibroblast                                         2
               low quality                                        1
TSP8  Prostate cd8-positive, alpha-beta t cell                   91
               nkt cell                                           2
TSP9  Pancreas t cell                                             7

[202 rows x 1 columns]

In [31]:
merged.to_csv('../data/merged_airr.tsv', sep = '\t', index=False)